In [1]:
import os
import sys
import xml.etree.ElementTree as ElementTree
import collections
from collections import Counter
import pickle
import json

import psycopg2

import parse_article

In [2]:
dossier_data = '/home/michel/loi/data/'

## Charge les infos

In [3]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

curseur.execute("select base_origine, categorie, cid, id_ from article where valide = True;")
liste_article = curseur.fetchall()

curseur.close()
connection.close()

In [5]:
valeurs_par_id_ = {}

i = 0
for base_origine, categorie, cid, id_ in liste_article:
    nom_fichier = dossier_data + 'parse_article/' + id_

    with open(nom_fichier, 'rb') as f:
        valeurs = pickle.load(f)
        
        valeurs_par_id_[id_] = valeurs
        
        i += 1
        if i % 100000 == 0:
            print(i)


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
1700000
1800000
1900000
2000000
2100000
2200000
2300000
2400000
2500000
2600000
2700000
2800000
2900000
3000000
3100000


## Traitements

Création de la nouvelle table contenant les articles :
```
create table article_2 (
    base_origine varchar(21),
    categorie varchar(21),
    cid varchar(21),
    id_ varchar(21),
    valide boolean,

    ID_ELI text,
    ID_ELI_ALIAS text,
    ANCIEN_ID text,
    ORIGINE text,
    URL text,
    NATURE text,
    NUM text,
    MCS_ART text,
    ETAT text,
    DATE_DEBUT text,
    DATE_FIN text,
    TYPE text,
    TEXTE text,
    TITRES_TXT text,
    TMS text,
    VERSIONS text,
    NOTA text,
    SM text,
    BLOC_TEXTUEL text,
    LIENS text
);
create index on article_2 (id_);
```

In [20]:
def serialisable(objet):
    if isinstance(objet, parse_article.FrozenDict):
        return {c: serialisable(v) for c, v in objet.items()}
    elif isinstance(objet, frozenset):
        return [serialisable(e) for e in objet]
    elif isinstance(objet, tuple):
        return [serialisable(e) for e in objet]
    else:
        return objet


def traite_id_(base_origine, categorie, cid, id_):
    valeurs = valeurs_par_id_[id_]
    assert valeurs['ID'] == id_

    valeurs_serialisables = {
        'base_origine': base_origine,
        'categorie': categorie,
        'cid': cid,
        'id_': id_,
        'valide': True,

        'ID_ELI': valeurs['ID_ELI'],
        'ID_ELI_ALIAS': valeurs['ID_ELI_ALIAS'],
        'ANCIEN_ID': valeurs['ANCIEN_ID'],
        'ORIGINE': valeurs['ORIGINE'],
        'URL': valeurs['URL'],
        'NATURE': valeurs['NATURE'],
        'NUM': valeurs['NUM'],
        'MCS_ART': valeurs['MCS_ART'],
        'ETAT': valeurs['ETAT'],
        'DATE_DEBUT': valeurs['DATE_DEBUT'],
        'DATE_FIN': valeurs['DATE_FIN'],
        'TYPE': valeurs['TYPE'],
        'NOTA': valeurs['NOTA'],
        'SM': valeurs['SM'],
        'BLOC_TEXTUEL': valeurs['BLOC_TEXTUEL'],
    }

    valeurs_serialisables['TEXTE'] = json.dumps(dict(valeurs['TEXTE']))
    valeurs_serialisables['TITRES_TXT'] = json.dumps([dict(fd) for fd in valeurs['TITRES_TXT']])
    valeurs_serialisables['VERSIONS'] = json.dumps([dict(fd) for fd in valeurs['VERSIONS']])
    valeurs_serialisables['LIENS'] = json.dumps([dict(fd) for fd in valeurs['LIENS']])
    valeurs_serialisables['TMS'] = json.dumps(serialisable(valeurs['TMS']))

    return valeurs_serialisables


In [32]:
connection = psycopg2.connect(dbname='loi', user='loi', password='baba')
curseur = connection.cursor()

In [27]:
valeurs = []
for base_origine, categorie, cid, id_ in liste_article:
    valeurs.append(traite_id_(base_origine, categorie, cid, id_))

In [28]:
liste_champ = list(valeurs[0].keys())

In [29]:
liste_chaine = [
    curseur.mogrify("(" + ','.join(["%s"] * len(liste_champ)) + ")",
            [v[c] for c in liste_champ]
        )
    for v in valeurs
]


In [33]:
nb_fournee = 1000
for i_fournee in range(len(liste_chaine)//nb_fournee + 1):
    sous_liste = liste_chaine[i_fournee*nb_fournee:(i_fournee+1)*nb_fournee]
    chaine_valeurs = b','.join(sous_liste)
    
    curseur.execute(b"insert into article_2 (" + b','.join([bytearray(c, 'utf-8') for c in liste_champ]) + b") values " + chaine_valeurs + b";")
    connection.commit()

In [34]:
curseur.close()
connection.close()